<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-1/Insumos/Acc_Div_SMMC_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Montecarlo)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [12]:
!pip install yfinance
import yfinance as yf
import pandas as pd


In [13]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [14]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))
DeltaX=DeltaX.dropna()

print(DeltaX.head())
print(X0)


                 AAPL         AMZN        MSFT         SQ
Date                                                     
2020-01-02  75.087502  1898.010010  160.619995  63.830002
2020-01-03  74.357498  1874.969971  158.619995  63.000000
2020-01-06  74.949997  1902.880005  159.029999  62.570000
2020-01-07  74.597504  1906.859985  157.580002  64.589996
2020-01-08  75.797501  1891.969971  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
2020-01-09  0.021018  0.004788  0.012416  0.004870
                  AAPL         AMZN        MSFT         SQ
Date                                                      
2022-05-12  142.559998  2138.610107  255.350006  75.760002


In [15]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    1.425600e+05
AMZN    1.069305e+07
MSFT    5.107000e+05
SQ      7.576000e+03
Name: 2022-05-12 00:00:00, dtype: float64

## Simulación

	
		Paso 1. Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$ (		donde $T_1(x)=\ln(x)$ ó $T_2(x)=x-1$), es decir
		$${\Delta X}_\tau=\Big[ T_j(\frac{x_{t,1}}{x_{t+\tau,1}}), T_j(\frac{x_{t,2}}{x_{t+\tau,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+\tau,m}})\Big]\quad t=0,1,\ldots, n-1.$$
		Paso 2. Calcular la matriz de correlación de ${\Delta X}_\tau$, i.e. $\Omega=\Corr({\Delta X}_\tau)$.
		Paso 3. Factorizar $\Omega=AA^\top$.
		Paso 4. Analizar la distribución individual de las variaciones de los factores de riesgo ${\Delta X}_{\tau,k}$ ($k=1,\ldots,m$) para ajustar a una función de distribución paramétrica conocida ó utilizar la función empírica tal cual.
		Paso 5. Calcular matriz $S$ de $N\times m$ valores con $$S_{i\cdot}=\Big[F^{-1}_{{\Delta X}_{\tau,1}}(\alpha_{i 1}),\ldots,F^{-1}_{{\Delta X}_{\tau,m}}(\alpha_{i m})\Big]$$ donde $\alpha_{i\cdot}\sim U(0,1)$ con $N\times m$ valores a.i.i.d. simulados, y $F^{-1}_{{\Delta X}_{\tau,k}}$ es la inversa de la distribución empírica o paramétrica (seleccionada en el paso anterior) de las variaciones del factor de riesgo $k$, i.e. de ${\Delta X}_{\tau,k}$ ($k=1,\ldots,m$).
		Paso 6. Calcular matriz $\Delta X_{s_\tau}$ como $\Delta X_{s_\tau}=S\cdot A^\top$. 
		Paso 7. Construir ${X}_{s_{N\times m}}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{s_{i\cdot}}=\Big[x_{0,1}T^{-1}_j(\Delta x_{s_{i1}}), \ldots, x_{0,m}T^{-1}_j(\Delta x_{s_{im}})\Big]\quad (i=1,\ldots,N\quad j=1\, \textrm{ó}\, 2),$$

Paso 1. Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$ (		donde $T_1(x)=\ln(x)$ ó $T_2(x)=x-1$), es decir
		$${\Delta X}_\tau=\Big[ T_j(\frac{x_{t,1}}{x_{t+\tau,1}}), T_j(\frac{x_{t,2}}{x_{t+\tau,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+\tau,m}})\Big]\quad t=0,1,\ldots, n-1.$$
		Paso 2. Calcular la matriz de correlación de ${\Delta X}_\tau$, i.e. $\Omega=\Corr({\Delta X}_\tau)$.

Paso 3. Factorizar $\Omega=AA^\top$.

Paso 4. Analizar la distribución individual de las variaciones de los factores de riesgo ${\Delta X}_{\tau,k}$ ($k=1,\ldots,m$) para ajustar a una función de distribución paramétrica conocida ó utilizar la función empírica tal cual.

Paso 5. Calcular matriz $S$ de $N\times m$ valores con $$S_{i\cdot}=\Big[F^{-1}_{{\Delta X}_{\tau,1}}(\alpha_{i 1}),\ldots,F^{-1}_{{\Delta X}_{\tau,m}}(\alpha_{i m})\Big]$$ donde $\alpha_{i\cdot}\sim U(0,1)$ con $N\times m$ valores a.i.i.d. simulados, y $F^{-1}_{{\Delta X}_{\tau,k}}$ es la inversa de la distribución empírica o paramétrica (seleccionada en el paso anterior) de las variaciones del factor de riesgo $k$, i.e. de ${\Delta X}_{\tau,k}$ ($k=1,\ldots,m$).

Paso 6. Calcular matriz $\Delta X_{s_\tau}$ como $\Delta X_{s_\tau}=S\cdot A^\top$.

Paso 7. Construir ${X}_{s_{N\times m}}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{s_{i\cdot}}=\Big[x_{0,1}T^{-1}_j(\Delta x_{s_{i1}}), \ldots, x_{0,m}T^{-1}_j(\Delta x_{s_{im}})\Big]\quad (i=1,\ldots,N\quad j=1\, \textrm{ó}\, 2),$$

In [23]:
Ns=10000 #Definimos número de simulaciones
alpha=0.98 #Nivel de Confianza para las medidas de riesgo
ChCP=0  #0 Cholesky, 1 Componentes Principales
normempi=0 #0 normal, 1 empírico
eta=.85 #Varianza explicada  (sólo aplica para Componentes Principales)
DeltaX_port=DeltaX
mT=DeltaX_port.shape[1]
print(mT)

print(np.cov(DeltaX_port.T)*(DeltaX_port.shape[1]/(DeltaX_port.shape[1]-1)))

print(((DeltaX_port.shape[1])/(DeltaX_port.shape[1]-1)))


VarDeltaX=np.where(ChCP==0,np.corrcoef(DeltaX_port.T),np.cov(DeltaX_port.T)*((DeltaX_port.shape[1])/(DeltaX_port.shape[1]-1)))


if(ChCP==0):
  CVarDeltaX=np.linalg.cholesky(VarDeltaX)
  M=np.zeros([Ns, mT])
  print((CVarDeltaX.shape[1]))
  print(VarDeltaX.shape[1])
  if normempi==1:
    for i in range(mT):
      M[:,i]=DeltaX_port.iloc[:,i].sort_values(ascending=True).quantile(1-np.random.uniform(0,1,Ns), interpolation = 'higher')
    Met='Cholesky Empírico'
  else:
    for i in range(mT):
      M[:,i]=np.random.normal(np.mean(DeltaX_port.iloc[:,i]), np.std(DeltaX_port.iloc[:,i]), size=(Ns))
    Met='Cholesky Normal'
  DeltaX_s=M.dot(CVarDeltaX) 
else:
  eig = np.linalg.eigh(VarDeltaX)
  valoresP = np.flip(eig[0])
  vectoresP = np.flip(eig[1])
  varacum = valoresP.cumsum()/valoresP.sum()
  n_comp = list(varacum).index(varacum[varacum>=eta][0])
  Y = (DeltaX_port - np.mean(DeltaX_port, axis = 0)).dot(vectoresP[:,0:n_comp+1])
  print(Y.shape)
  M=np.zeros([Ns, n_comp+1])

  # Simulación
  if normempi==1:
    for i in range(n_comp+1):
      M[:,i]=Y.iloc[:,i].sort_values(ascending=True).quantile(1-np.random.uniform(0,1,Ns), interpolation = 'higher')
    Met='PCA Empírico'
  else:
    for i in range(n_comp+1):
      # Aquí hay un problema
      M[:,i]=np.random.normal(np.mean(Y.iloc[:,i]), np.std(Y.iloc[:,i]), size=(Ns))
    Met='PCA Normal'
  DeltaX_s=M.dot(vectoresP[:,0:n_comp+1].T)


Met


Ms1='Se explica el' 
Ms2= 'de varianza'
Ms3= 'de suma de var-cov'

print(Ms1)
print(sum(np.diag(np.cov(DeltaX_s.T)))/sum(np.diag(np.cov(DeltaX_port.T)))) #Comprobación de matriz de varianza-cov vs original
print(Ms2)

print(Ms1)
print((np.ones(DeltaX_s.shape[1]).dot(np.cov(DeltaX_s.T)).dot(np.ones(DeltaX_s.shape[1]).T))/(np.ones(DeltaX_s.shape[1]).dot(np.cov(DeltaX_port.T)).dot(np.ones((DeltaX_s.shape[1]))))) #Comprobación de matriz de varianza-cov vs original
print(Ms3)

print(mT)
print(n_comp) #número de variables simuladas  

4
[[0.00072389 0.00045961 0.00054918 0.00078142]
 [0.00045961 0.00069512 0.00045514 0.00075333]
 [0.00054918 0.00045514 0.0006378  0.00079975]
 [0.00078142 0.00075333 0.00079975 0.0026927 ]]
1.3333333333333333
4
4
Se explica el
0.9896695333077943
de varianza
Se explica el
0.9706789033334196
de suma de var-cov
4
1



Paso 8. Construcción de la matriz de reevaluación del portafolio basada en los escenarios simulados $Y_{N\times r}$ de todos los instrumentos financieros, es decir
$$Y_z=[m_1f_1(X_{s_z}), m_2f_2(X_{s_z}),\ldots, m_rf_r(X_{s_z})]\quad (z=1,\ldots,N).$$
Paso 9. Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios simulados $\Delta Y_{N\times r}$ de todos los instrumentos financieros, es decir
$$\Delta Y_{z\cdot}=Y_0-Y_{z}=[m_1(f_1(X_{00})-f_1(X_{s_{z\cdot}})),\ldots, m_r(f_r(X_{00})-f_r(X_{s_{z\cdot}}))],$$
para $(z=1,\ldots,N)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{N}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{z\cdot}$ $(z=1,\ldots,N)$, incluso se puede hacer lo mismo por tipo de riesgo, ya que para cada tipo de riesgo pueden existir diferentes instrumentos, lo único que se tiene que hacer es sumar los instrumentos de un mismo tipo de riesgo por escenario.
Paso 10. Obtener las medidas de riesgo deseada de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$, ejemplo se puede calcular $\VaR_\alpha({\Delta Y}_{\cdot k})$, $\CVaR_\alpha({\Delta Y}_{\cdot k})$, $\CeR_\alpha({\Delta Y}_{\cdot k})$, $\mu(\Delta Y_{\cdot k})$, $\sigma(\Delta Y_{\cdot k})$, etc. para $k=1,\ldots,r$. 



In [76]:
print(DeltaX_s.shape)
print(X0.shape)
print(X0.T.squeeze().T)

X_s=np.exp(pd.DataFrame(DeltaX_s).squeeze())*np.array(X0.squeeze())
X_s.head

(10000, 4)
(1, 4)
AAPL     142.559998
AMZN    2138.610107
MSFT     255.350006
SQ        75.760002
Name: 2022-05-12 00:00:00, dtype: float64


<bound method NDFrame.head of                0            1           2          3
0     150.382675  2168.990345  258.003884  76.871107
1     139.654715  2137.510707  256.992507  78.773486
2     137.967185  2100.432758  251.723302  75.026633
3     139.040343  2132.555361  250.292220  72.038264
4     152.453051  2140.034526  257.214299  78.837977
...          ...          ...         ...        ...
9995  151.926342  2134.931610  259.449436  77.767081
9996  144.135229  2190.048220  252.456898  74.872178
9997  141.027135  2144.250299  255.049176  75.207531
9998  150.135182  2190.765439  262.269476  81.748660
9999  138.559012  2111.585530  256.251075  77.667438

[10000 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [77]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of              0          1         2         3
0     4.081135  19.573303  3.132681  1.458767
1    -4.755681 -33.021594 -1.584196  4.004831
2     0.080530 -28.565697 -5.426308 -0.951221
3     1.221455  45.273579 -3.341905 -4.729708
4     7.851593 -31.189011 -1.278415  4.092201
...        ...        ...       ...       ...
9995  6.452279 -38.402953  4.411592  2.650348
9996  0.835456  88.487712 -3.869137 -1.150370
9997 -1.476631  13.332487  0.306277 -0.717450
9998 -1.820163  -9.442525  3.630866  8.106213
9999 -3.583701 -54.543346 -1.270058  2.517161

[10000 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [79]:
PL=Y_s-np.array(Y0.squeeze())

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                   0             1              2            3         Total
0    -142555.916424 -1.069303e+07 -510696.879526 -7574.541447 -1.135386e+07
1    -142564.753240 -1.069308e+07 -510701.596403 -7571.995382 -1.135392e+07
2    -142559.917029 -1.069308e+07 -510705.438515 -7576.951434 -1.135392e+07
3    -142558.776104 -1.069301e+07 -510703.354112 -7580.729922 -1.135385e+07
4    -142552.145966 -1.069308e+07 -510701.290622 -7571.908013 -1.135391e+07
...             ...           ...            ...          ...           ...
9995 -142553.545279 -1.069309e+07 -510695.600615 -7573.349866 -1.135391e+07
9996 -142559.162103 -1.069296e+07 -510703.881344 -7577.150584 -1.135380e+07
9997 -142561.474190 -1.069304e+07 -510699.705930 -7576.717664 -1.135388e+07
9998 -142561.817721 -1.069306e+07 -510696.381341 -7567.894001 -1.135389e+07
9999 -142563.581259 -1.069311e+07 -510701.282265 -7573.483052 -1.135394e+07

[10000 rows x 5 columns]>

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [80]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [85]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)


VaR=[]

for i in range(0,PL.shape[1]):
  VC=PL.iloc[:,i].sort_values(ascending=True).quantile(1-alpha, interpolation = 'higher')
  VaR.append(VC)

print(VaR)

print(sum(Y0))

0       -1.425662e+05
1       -1.069315e+07
2       -5.107110e+05
3       -7.582508e+03
Total   -1.135398e+07
Name: 0.020000000000000018, dtype: float64
[-142566.21817564737, -10693145.555554422, -510711.0374065455, -7582.508213973009, -11353981.438475251]
11353886.547088623


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [82]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [86]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [92]:

print(VaR)
print(VaR_C)



[-142566.21817564737, -10693145.555554422, -510711.0374065455, -7582.508213973009, -11353981.438475251]
[-142567.29029850083, -10693161.472683161, -510712.88864578586, -7583.6059374901915, -11353997.434505552]
